
### Librerías

In [1]:
import sys
import pandas as pd
import GetOldTweets3 as got
import sys
import matplotlib.pyplot as plt
from googletrans import Translator
import socket as sk
from datetime import *
import tweepy
import re
import string
from textblob import TextBlob
import nltk
from wordcloud import WordCloud

### Configuración de parámetros 

In [2]:
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000
sk.setdefaulttimeout(600)

#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

### Ejecución de clases y métodos del proyecto

In [12]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb
%run GetOldTweets.ipynb
%run NLP.ipynb
%run Extract_Twitter_v2.ipynb

# Método Principal - TFM Data Science

In [ ]:
if __name__ == '__main__':
    
    #Control horario programa
    fecha_inicio = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo del programa: ',fecha_inicio)
    
    # Definición de variables
    fichero = 'CSVs/info_tweets_el_corte_ingles_202003.csv'
    pattern = 'CSVs/info_tweets_el_corte_ingles*.csv'
    fichero_user = 'CSVs/following_info_el_corte_ingles.csv'
    fichero_tweets = 'CdM/Tweets_Info.csv'
    fichero_fact = 'CdM/Tweets_Fact_Info.csv'
    following_ids = []  
    calendario = calendar()
    df_tweets, df_tweets_final, df_following, df_fact_following = creacion_DataFrame()
    #positivo, negativo, neutral, sentiment = creacion_Variables()
    
    #Flujo de la información
    online = 0
    all_tweets = 0
    online_user = 0
    
    #Definición POO
    file = File_Manage()
    #file.initValues(fichero, pattern)
    gotTweets = GoT(calendario)
    nlp = NLP()
    google = Translator()
    
    #Acceso a Twitter
    api = acceso_Twitter() 

    
    #Obtención por GetOldTweets
    if online == 1:
        print('Opción Online')
        print(gotTweets.getCalendar())
        df_tweets = gotTweets.get_Tweets(gotTweets.getCalendar())
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    elif all_tweets == 1:
        file.setPattern(pattern)
        print('Opción Offline Tweets: ', file.getPattern())
        df_tweets = file.read_all_Tweets(file.getPattern()) 

    else: 
        file.setFile(fichero)
        print('Opción Un fichero: ', file.getFile())
        df_tweets = file.read_Tweets(file.getFile())
        
    df_tweets['valor'] = -10
    df_tweets['fecha'] = df_tweets['date'].astype(str).str[:10]
    df_tweets['fecha'] = df_tweets['fecha'].str.replace('-','/')
    df_tweets['objetividad'] = 0
    df_tweets['subjetividad'] = 0

    df_tweets['Tweet_punct'] = df_tweets['text'].str.lower().apply(lambda text: nlp.eliminar_signos(text))
    df_tweets['Tweet_traduccion'] = df_tweets['Tweet_punct'].apply(lambda text: google.translate(text,dest='es').text)
    df_tweets['Tweet_token'] = df_tweets['Tweet_traduccion'].apply(lambda text: nlp.tokenization(text))
    df_tweets['Tweet_novacias'] = df_tweets['Tweet_token'].apply(lambda text: nlp.quitar_palabras_vacias(text))
    df_tweets['Tweet_novacias'] = df_tweets['Tweet_novacias'].apply(lambda text: listToString(text))

    
    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    df_tweets_final = nlp.analisis(df_tweets)

    nlp.nube_palabras(df_tweets_final)

    
    #Generación de CSVs para el CdM
    file.setFile(fichero_tweets)
    #file.writeCSV(df_tweets_final,file.getFile())
    
    #Para el caso de los usuarios followings, se crea un minimodelo de datos
    if online_user == 1:
        following_ids = get_following_ids(api)
        following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
        df_following = pd.DataFrame(get_info_following(api,following_limit))
        file.setFile(fichero_user)
        file.writeCSV(df_following,file.getFile())
    else:
        file.setFile(fichero_user)
        df_following = file.readCSV(df_following,file.getFile())
     
    df_tweets.columns = ['id_text','created_at_tweet','name_account','text','geo','valor','fecha','objetividad','subjetividad','Tweet_punct','Tweet_traduccion','Tweet_token','Tweet_novacias']

    df_following.columns = ['id_account','name_account','screen_name_account','created_at_account','location_account','followers_account','friends_account','description_account']
    df_fact_following = pd.merge(df_tweets, df_following,on=['name_account'],how='left')
    df_fact_following = df_fact_following[df_fact_following['id_account'].notnull()]
        
    
    file.setFile(fichero_fact)    
    #file.writeCSV(df_fact_following,file.getFile())
    
    fecha_fin = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Hora de Finalizacion: ',fecha_fin)

Comienzo del programa:  08/12/2020, 20:32:52
Opción Un fichero:  CSVs/info_tweets_el_corte_ingles_202003.csv


In [ ]:
df_tweets.head(100)